In [2]:
# -*- coding: utf-8 -*-
from gemm_tiling import gemm_tiling_input_stationary, gemm_tiling_weight_stationary
import transformer_block as tbk
import arch_execution as arch
from util import *
import math
from mapper import *

In [3]:
llm_config = load_config("./input/transformer/input0.json")
model = tbk.Llama_block(llm_config)
tx8_config = load_config('hardware_parameter.json')
hardware = arch.Tx8(tx8_config)
Layers = model.config['L']
ops = model.ops
mapping_result = {}

通过tuning不同的tile_m，tile_n可以得到不同的性能结果

因为 M=K=N，所以input stationary和output stationary 可以设置相反的tile_m和tile_n，从而得到完全一样的性能

下面分析 q、k、v是否融合的影响

In [6]:
# =============== QKV projection  ===================
M, K, N = 4096, 4096, 4096
QKV = 3   # QKV = 3, if fuse q k v into one matrix
N = N * QKV
details = False
B = 1
print("QKV projection, QKV=3, fuse q k v into one matrix")
tile_m = 64
tile_n = 64
utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = True
ops["QKV_fusion"] = model.gen_gemm(
    "QKV_fusion", [B, model.config["S"], model.config["H"], model.config["H"]])
mapping_result['QKV_fusion'] = gemm_auto_opt_mapper(
    ops['QKV_fusion'], hardware, Tm_Tn=Tm_Tn, fusion_op1=ops['RMSNorm'], details=details)
utilization = mapping_result['QKV_fusion']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")


tile_m = 128
tile_n = 32
utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = True
ops["QKV_fusion"] = model.gen_gemm(
    "QKV_fusion", [B, model.config["S"], model.config["H"], model.config["H"]])
mapping_result['QKV_fusion'] = gemm_auto_opt_mapper(
    ops['QKV_fusion'], hardware, Tm_Tn=Tm_Tn, fusion_op1=ops['RMSNorm'], details=details)
utilization = mapping_result['QKV_fusion']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")


tile_m = 256
tile_n = 32
utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = True
ops["QKV_fusion"] = model.gen_gemm(
    "QKV_fusion", [B, model.config["S"], model.config["H"], model.config["H"]])
mapping_result['QKV_fusion'] = gemm_auto_opt_mapper(
    ops['QKV_fusion'], hardware, Tm_Tn=Tm_Tn, fusion_op1=ops['RMSNorm'], details=details)
utilization = mapping_result['QKV_fusion']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")


tile_m = 32
tile_n = 128
utilization = gemm_tiling_weight_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: weight, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = False
ops["QKV_fusion"] = model.gen_gemm(
    "QKV_fusion", [B, model.config["S"], model.config["H"], model.config["H"]])
mapping_result['QKV_fusion'] = gemm_auto_opt_mapper(
    ops['QKV_fusion'], hardware, Tm_Tn=Tm_Tn, fusion_op1=ops['RMSNorm'], details=details)
utilization = mapping_result['QKV_fusion']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

QKV projection, QKV=3, fuse q k v into one matrix
QKV projection, M=4096, K=4096, N=12288, B=1, tile_m=64, tile_n=64, stationary: input, utilization=82.56%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[64, 64], Hardware utilization: 79.69%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
QKV projection, M=4096, K=4096, N=12288, B=1, tile_m=128, tile_n=32, stationary: input, utilization=91.84%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[128, 32], Hardware utilization: 83.23%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
QKV projection, M=4096, K=4096, N=12288, B=1, tile_m=256, tile_n=32, stationary: input, utilization=90.09%
输入不复用则SRAM满足要求
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[256, 32], Hardware utilization: 79.35%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
QKV projection, M=4096, K=4096, N=12288, B=1, tile_m=32, tile_n=128, stationary: weight, utilization=83.36%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[32, 128], Hardware utilization: 0.00%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


可见，对于weight stationary来说，fuse qkv没太大变化，但是对于input stationary来说，fuse qkv会有很大的提升，因为提高了input 的复用率。

下面分析batch size的影响

In [7]:
# =============== QKV projection  ===================
M, K, N = 4096, 4096, 4096
QKV = 1   # QKV = 3, if fuse q k v into one matrix
N = N * QKV
B = 16
ops['K_proj']['ishape'][0] = B
ops['K_proj']['oshape'][0] = B
print(f"Batch size = {ops['K_proj']['ishape'][0]}")
print(f"QKV projection, batch size analysis, B={B}")
tile_m = 64
tile_n = 64
utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
ops['K_proj']['ishape'][0] = ops['K_proj']['ishape'][0]
op = ops['K_proj']
print(f"op input shape = {op['ishape']}")
print(f"op output shape = {op['oshape']}")
mapping_result['K_proj'] = gemm_auto_opt_mapper(
    ops['K_proj'], hardware, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['K_proj']["utilization"]*100
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")


tile_m = 128
tile_n = 32

utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
print(f"Batch size = {ops['K_proj']['ishape'][0]}")

mapping_result['K_proj'] = gemm_auto_opt_mapper(
    ops['K_proj'], hardware, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['K_proj']["utilization"]*100
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")


tile_m = 256
tile_n = 32
utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
print(f"Batch size = {ops['K_proj']['ishape'][0]}")

mapping_result['K_proj'] = gemm_auto_opt_mapper(
    ops['K_proj'], hardware, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['K_proj']["utilization"]*100
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

tile_m = 32
tile_n = 128
utilization = gemm_tiling_weight_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(
    f"QKV projection, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: weight, utilization={utilization:.2f}%")


Tm_Tn = [int(tile_m), int(tile_n)]
print(f"Batch size = {ops['K_proj']['ishape'][0]}")

mapping_result['K_proj'] = gemm_auto_opt_mapper(
    ops['K_proj'], hardware, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['K_proj']["utilization"]*100
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

Batch size = 16
QKV projection, batch size analysis, B=16
QKV projection, M=4096, K=4096, N=4096, B=16, tile_m=64, tile_n=64, stationary: input, utilization=85.17%
op input shape = [16, 4096, 4096]
op output shape = [16, 4096, 4096]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[64, 64], Hardware utilization: 5.20%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
QKV projection, M=4096, K=4096, N=4096, B=16, tile_m=128, tile_n=32, stationary: input, utilization=85.29%
Batch size = 16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[128, 32], Hardware utilization: 7.57%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
QKV projection, M=4096, K=4096, N=4096, B=16, tile_m=256, tile_n=32, stationary: input, utilization=84.89%
Batch size = 16
输入不复用则SRAM满足要求
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Tm_Tn=[256, 32], Hardware utilization: 11.05%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
QKV projection, M=4096, K=4096, N=4096, B=16, tile_m=32, tile_n=128, stationary: weight, utilization=98.36%
Batch size = 16
$$$$$$$$$$$$$$$$$$$